In [1]:
import sys
sys.path.append('../engine/')

from DataManager import DataManager
from Hybrid import Hybrid
import rpy2.robjects.packages as rpackages


num_bootstraps = 10
num_folds = 5

fs_methods = [
    ("reliefF", "python", "rf"),
    ("geoDE", "python", "gd")
]

ths = [1, 5, 10, 15, 25, 50, 75, 100, 150, 200]
seed = 42

In [2]:
dataset_path = '/home/colombelli/Documents/datasets/thyroid_log2.rds'
results_path = '/home/colombelli/Desktop/thyroid/'

In [3]:
def select_features_heavy(self):
    
        for i in range(self.dm.num_folds):
            
            print("\n\n################# Fold iteration:", i+1, "#################")
            self.dm.current_fold_iteration = i
            self.dm.update_bootstraps()

            bs_rankings = {}
            for j, (bootstrap, _) in enumerate(self.dm.current_bootstraps):
                print("\n\nBootstrap: ", j+1, "\n")
                output_path = self.dm.get_output_path(i, j)
                bootstrap_data = self.dm.pd_df.loc[bootstrap]

        
                fst_layer_rankings = []
                for fs_method in self.fs_methods:   
                    print("")
                
                bs_rankings[j] = fst_layer_rankings
                self.dm.bs_rankiings = bs_rankings
            print("\nAggregating Level 1 rankings...")
            return bs_rankings

In [15]:
dm = DataManager(results_path, dataset_path, num_bootstraps, num_folds, seed)
dm.encode_main_dm_df()
dm.create_results_dir()
dm.init_data_folding_process()

Encoding dataframe attributes...
Creating results directory...


In [16]:
aggregator = "mean"
ensemble = Hybrid(dm, fs_methods, aggregator, aggregator, ths)

In [17]:
ensemble.select_features = select_features_heavy

In [92]:
bs_rankings = ensemble.select_features(ensemble)



################# Fold iteration: 1 #################


Bootstrap:  1 


otp path: /home/colombelli/Desktop/thyroid/fold_1/bootstrap_1/
Ranking features with ReliefF algorithm...
Processing data...
Decoding dataframe attributes...
Saving ranking...

otp path: /home/colombelli/Desktop/thyroid/fold_1/bootstrap_1/
Ranking features with Characteristic Direction (GeoDE)...
Processing data...
Decoding dataframe attributes...
Saving ranking...


Bootstrap:  2 


otp path: /home/colombelli/Desktop/thyroid/fold_1/bootstrap_2/
Ranking features with ReliefF algorithm...
Processing data...
Decoding dataframe attributes...
Saving ranking...

otp path: /home/colombelli/Desktop/thyroid/fold_1/bootstrap_2/
Ranking features with Characteristic Direction (GeoDE)...
Processing data...
Decoding dataframe attributes...
Saving ranking...


Bootstrap:  3 


otp path: /home/colombelli/Desktop/thyroid/fold_1/bootstrap_3/
Ranking features with ReliefF algorithm...
Processing data...
Decoding dataframe attribu

In [19]:
bs_rankings[0]

[                       rank
 Xhsa_2dmiR_2d409_2d3p     1
 Xhsa_2dmiR_2d592          2
 Xhsa_2dmiR_2d433          3
 Xhsa_2dmiR_2d409_2d5p     4
 Xhsa_2dmiR_2d485_2d5p     5
 ...                     ...
 Xebv_2dmiR_2dBART8      813
 Xhsa_2dmiR_2d613        814
 Xhsa_2dmiR_2d615_2d5p   815
 Xhsa_2dmiR_2d504        816
 Xhsa_2dmiR_2d553        817
 
 [817 rows x 1 columns],                         rank
 Xhsa_2dmiR_2d142_2d5p      1
 Xhsa_2dmiR_2d31            2
 Xhsa_2dmiR_2d210           3
 Xhsa_2dmiR_2d193b          4
 Xhsa_2dmiR_2d205           5
 ...                      ...
 Xhsa_2dmiR_2d639         813
 Xebv_2dmiR_2dBART8       814
 Xhsa_2dmiR_2d99b_2A      815
 Xhcmv_2dmiR_2dUS25_2d1   816
 Xhsa_2dmiR_2d302b        817
 
 [817 rows x 1 columns]]

In [83]:
import kuncheva_index as ki

def get_fs_stabilities(threshold, bs_rankings):
    
    num_fs = len(bs_rankings[0])
    
    stabilities = []
    for fs in range(num_fs):
        
        fs_rankings = []
        for bs in bs_rankings:
            df_ranking = bs_rankings[bs][fs]
            lst_genes = list(df_ranking.index.values)
            fs_rankings.append(lst_genes)
        
        stabilities.append(ki.get_kuncheva_index(fs_rankings, threshold=threshold))
    
    return stabilities

In [112]:
import pandas as pd


def build_df_correct_order(aggregated_ranking):
    
    final_ranking = pd.DataFrame.from_dict(aggregated_ranking, orient='index')
    final_ranking.columns = ['rank']
    final_ranking = final_ranking.sort_values(by='rank', ascending=False)
    final_ranking.iloc[:] = final_ranking.iloc[::-1].values
    return final_ranking

In [37]:
ths = [1, 5, 10, 15, 25, 50, 75, 100, 150, 200]
sum(ths) // len(ths)

63

In [41]:
get_selectors_stability(63, bs_rankings)

[0.4114025608278481, 0.3445164881946492]

In [113]:
def aggregate():
    mean_th = sum(ths) // len(ths)
    fs_stabilities = get_fs_stabilities(mean_th, bs_rankings)
    
    aggregated_ranking = {}
    
    for gene in list(bs_rankings[0][0].index):
        aggregated_ranking[gene] = 0


    for bs in bs_rankings:
        for fs, ranking in enumerate(bs_rankings[bs]):
            reversed_ranking = ranking.iloc[::-1]
            for gene in reversed_ranking.index: 
                aggregated_ranking[gene] += (reversed_ranking.index.get_loc(gene)+1) * fs_stabilities[fs]
    
    
    return build_df_correct_order(aggregated_ranking)

In [45]:
from copy import deepcopy as dp

testdf = dp(bs_rankings[0][0])
testdf

,rank
Xhsa_2dmiR_2d409_2d3p,1
Xhsa_2dmiR_2d592,2
Xhsa_2dmiR_2d433,3
Xhsa_2dmiR_2d409_2d5p,4
Xhsa_2dmiR_2d485_2d5p,5
...,...
Xebv_2dmiR_2dBART8,813
Xhsa_2dmiR_2d613,814
Xhsa_2dmiR_2d615_2d5p,815
Xhsa_2dmiR_2d504,816


In [76]:
reversed_ranking = testdf.iloc[::-1]

In [72]:
testdf.index.get_loc('Xhsa_2dmiR_2d409_2d3p')

0

,rank
Xhsa_2dmiR_2d553,817
Xhsa_2dmiR_2d504,816
Xhsa_2dmiR_2d615_2d5p,815
Xhsa_2dmiR_2d613,814
Xebv_2dmiR_2dBART8,813
...,...
Xhsa_2dmiR_2d485_2d5p,5
Xhsa_2dmiR_2d409_2d5p,4
Xhsa_2dmiR_2d433,3
Xhsa_2dmiR_2d592,2


In [105]:
aggregate()

,rank
Xhsa_2dmiR_2d592,6362.307113
Xhsa_2dmiR_2d129_2d5p,6351.788151
Xhsa_2dmiR_2d124,6291.936465
Xhsa_2dmiR_2d376b,6275.857359
Xhsa_2dmiR_2d431_2A,6172.834205
...,...
XNC1_5f00000215,1144.388161
XNC2_5f00079215,1134.738239
XNC2_5f00092197,1127.521803
XNC2_5f00106057,1116.898486


In [100]:
import pandas as pd

In [111]:
aggregate()

,rank
Xhsa_2dmiR_2d592,1109.682050
Xhsa_2dmiR_2d129_2d5p,1116.898486
Xhsa_2dmiR_2d124,1127.521803
Xhsa_2dmiR_2d376b,1134.738239
Xhsa_2dmiR_2d431_2A,1144.388161
...,...
XNC1_5f00000215,6172.834205
XNC2_5f00079215,6275.857359
XNC2_5f00092197,6291.936465
XNC2_5f00106057,6351.788151


In [114]:
aggregate()

,rank
Xhsa_2dmiR_2d592,1109.682050
Xhsa_2dmiR_2d129_2d5p,1116.898486
Xhsa_2dmiR_2d124,1127.521803
Xhsa_2dmiR_2d376b,1134.738239
Xhsa_2dmiR_2d431_2A,1144.388161
...,...
XNC1_5f00000215,6172.834205
XNC2_5f00079215,6275.857359
XNC2_5f00092197,6291.936465
XNC2_5f00106057,6351.788151
